In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import seaborn
from datetime import datetime, timedelta

In [ ]:
font = {'family' : 'sans-serif',
        'weight' : 'bold',
        'size'   : 18}
matplotlib.rc('font', **font)
plt.rcParams["figure.figsize"] = [17, 7]
#plt.rcParams[""] = False

In [2]:
data_dir = 'Trace_files'
fnames = [x for x in filter(lambda x: x.endswith('.txt'), os.listdir(os.path.join('.', data_dir)))]
workload_names = [ x for x in map(lambda x: x.split('_')[1].split('.')[0], fnames) ]

In [3]:
# a dict of dataframes, tread (index) lightly..
col_names = ['startAddress', 'reqSize_sectors', 'reqSize_bytes', 'readWriteWaitFlag', 
              't_generated', 't_startProcess', 't_exitQueue', 't_finished']
dataset = {fname.split('_')[1].split('.')[0]:pd.read_csv(os.path.join(data_dir, fname), delimiter='\s+', header = None, dtype = float) for fname in fnames}
for wname in workload_names:
    dataset[wname].columns = col_names
# columns 4 through 7 contains the timestamps (floating point numbers), need to convert them into manipulable objects
for cindx in 4, 5, 6, 7:
    for df in dataset.values():
        # apply transformation (non destructive) to each column, then store the updated column back in the dataframe
        df.iloc[:,cindx] = df.iloc[:,cindx].apply(datetime.fromtimestamp)

### Sector being accessed more than once, implicitly..

In [4]:
dataset['radio'].head()

,startAddress,reqSize_sectors,reqSize_bytes,readWriteWaitFlag,t_generated,t_startProcess,t_exitQueue,t_finished
0,51707056.0,89.0,45568.0,5.0,2014-04-08 10:25:21.767840,2014-04-08 10:25:21.768273,2014-04-08 10:25:21.772367,2014-04-08 10:25:21.777534
1,51721376.0,281.0,143872.0,1.0,2014-04-08 10:25:21.768510,2014-04-08 10:25:21.772600,2014-04-08 10:25:21.777634,2014-04-08 10:25:21.791040
2,51865744.0,73.0,37376.0,1.0,2014-04-08 10:25:21.772781,2014-04-08 10:25:21.777782,2014-04-08 10:25:21.791130,2014-04-08 10:25:21.795952
3,4095328.0,104.0,53248.0,5.0,2014-04-08 10:25:22.176650,2014-04-08 10:25:22.176723,2014-04-08 10:25:22.180720,2014-04-08 10:25:22.184859
4,4095432.0,8.0,4096.0,5.0,2014-04-08 10:25:22.185035,2014-04-08 10:25:22.185142,2014-04-08 10:25:22.185153,2014-04-08 10:25:22.185846


In [ ]:
for wname in workload_names:
    dataset[wname]['reqSize_sectors_frombytes'] = dataset[wname]['reqSize_bytes'].apply(lambda x: x/512)

In [ ]:
dataset['radio'].iloc[:100,[0,-1]]